# Regra de associação

In [3]:
!pip install kagglehub[pandas-datasets]

In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [5]:
# Título do arquivo a ser descarregado
file_path = "wine-clustering.csv"

In [6]:
# Coleta de dados e armazenamento em dataframe Pandas
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS,"harrywang/wine-dataset-for-clustering",file_path)

/tmp/ipython-input-6-2079343211.py:2: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS,"harrywang/wine-dataset-for-clustering",file_path)


100%|██████████| 10.8k/10.8k [00:00<00:00, 3.83MB/s]


In [7]:
df.shape

(178, 13)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Alcohol               178 non-null    float64
 1   Malic_Acid            178 non-null    float64
 2   Ash                   178 non-null    float64
 3   Ash_Alcanity          178 non-null    float64
 4   Magnesium             178 non-null    int64  
 5   Total_Phenols         178 non-null    float64
 6   Flavanoids            178 non-null    float64
 7   Nonflavanoid_Phenols  178 non-null    float64
 8   Proanthocyanins       178 non-null    float64
 9   Color_Intensity       178 non-null    float64
 10  Hue                   178 non-null    float64
 11  OD280                 178 non-null    float64
 12  Proline               178 non-null    int64  
dtypes: float64(11), int64(2)
memory usage: 18.2 KB


In [9]:
df.head()

Alcohol  Malic_Acid   Ash  Ash_Alcanity  Magnesium  Total_Phenols  \
0    14.23        1.71  2.43          15.6        127           2.80   
1    13.20        1.78  2.14          11.2        100           2.65   
2    13.16        2.36  2.67          18.6        101           2.80   
3    14.37        1.95  2.50          16.8        113           3.85   
4    13.24        2.59  2.87          21.0        118           2.80   

   Flavanoids  Nonflavanoid_Phenols  Proanthocyanins  Color_Intensity   Hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   OD280  Proline  
0   3.92     1065  
1   3.40     1050  
2   3.17     1185  
3   3.45     1480  
4   2.93      735

In [10]:
# Transforma dados numéricos em categóricos

# Selecionar colunas numéricas principais
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

# Criar DataFrame para dados categorizados
df_categorical = pd.DataFrame()

# Transformar cada coluna numérica em categorias
for col in numeric_columns[:8]:  # Usar as primeiras 8 colunas (eleição aleatória, apenas para fins didáticos)
    # Dividir em 3 categorias: Baixo, Médio, Alto
    df_categorical[f'{col}_nivel'] = pd.cut(df[col],
                                           bins=3,
                                           labels=[f'{col}_BAIXO',
                                                  f'{col}_MEDIO',
                                                  f'{col}_ALTO'])

In [11]:
# Prepara dados para algoritmo Apriori
# Converter para formato de transações (lista de listas)
transactions = []

for index, row in df_categorical.iterrows():
    # Cada linha vira uma "transação" com os itens presentes
    transaction = [item for item in row.values if pd.notna(item)]
    transactions.append(transaction)

In [12]:
# Usar TransactionEncoder para formato adequado
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_transactions = pd.DataFrame(te_ary, columns=te.columns_)

print(f"Matriz de transações: {df_transactions.shape}")
print(f"Colunas (itens): {len(df_transactions.columns)} itens únicos")

Matriz de transações: (178, 24)
Colunas (itens): 24 itens únicos


In [13]:
# Aplicar Apriori para encontrar conjuntos de itens frequentes

frequent_itemsets = apriori(df_transactions, min_support=0.1, use_colnames=True) # min_support = 0.1 significa que o item deve aparecer em pelo menos 10% das transações

In [16]:
# Mostrar itens mais frequentes
top_itemsets = frequent_itemsets.nlargest(5, 'support')

for i, row in top_itemsets.iterrows():
    itemset = ', '.join(list(row['itemsets']))
    support_pct = row['support'] * 100
    print(f"  {support_pct:5.1f}% - {itemset}")

   70.2% - Ash_MEDIO
   64.0% - Ash_Alcanity_MEDIO
   64.0% - Malic_Acid_BAIXO
   56.2% - Magnesium_BAIXO
   50.6% - Flavanoids_MEDIO


In [ ]:
# Gerar regras com confiança mínima de 60%
rules = association_rules(frequent_itemsets,
                         metric="confidence",
                         min_threshold=0.6)

if len(rules) > 0:
    # Ordenar por confiança
    rules_sorted = rules.sort_values('confidence', ascending=False)

    print(f"TOP 3 REGRAS DE ASSOCIAÇÃO:")
    print("=" * 80)

    for i, row in rules_sorted.head(3).iterrows():
        antecedent = ', '.join(list(row['antecedents']))
        consequent = ', '.join(list(row['consequents']))
        confidence = row['confidence'] * 100
        support = row['support'] * 100
        lift = row['lift']

        print(f"REGRA {i+1}:")
        print(f"   SE: {antecedent}")
        print(f"   ENTÃO: {consequent}")
        print(f"   Confiança: {confidence:.1f}%")
        print(f"   Suporte: {support:.1f}%")
        print(f"   Lift: {lift:.2f}")

        # Interpretação do Lift
        if lift > 1.2:
            print(f"Correlação FORTE - Muito provável que ocorram juntos")
        elif lift > 1.0:
            print(f"Correlação MODERADA - Tendem a ocorrer juntos")
        else:
            print(f"Correlação FRACA - Independentes ou negativa")


🏆 TOP 3 REGRAS DE ASSOCIAÇÃO:

📋 REGRA 435:
   SE: Ash_MEDIO, Alcohol_MEDIO, Malic_Acid_MEDIO
   ENTÃO: Ash_Alcanity_MEDIO
   Confiança: 100.0%
   Suporte: 13.5%
   Lift: 1.56
Correlação FORTE - Muito provável que ocorram juntos

📋 REGRA 377:
   SE: Malic_Acid_BAIXO, Alcohol_ALTO, Ash_Alcanity_BAIXO
   ENTÃO: Ash_MEDIO
   Confiança: 100.0%
   Suporte: 11.2%
   Lift: 1.42
Correlação FORTE - Muito provável que ocorram juntos

📋 REGRA 507:
   SE: Total_Phenols_BAIXO, Alcohol_MEDIO, Malic_Acid_MEDIO
   ENTÃO: Flavanoids_BAIXO
   Confiança: 100.0%
   Suporte: 12.9%
   Lift: 2.25
Correlação FORTE - Muito provável que ocorram juntos
